In [1]:
import ast

class ComplexityVisitor(ast.NodeVisitor):
    def __init__(self):
        self.loop_depth = 0  
        self.max_loop_depth = 0  
        self.recursive_calls = 0
        self.function_name = None

    def visit_FunctionDef(self, node):
        """
        Visit a function definition to detect recursion.
        """
        self.function_name = node.name
        self.generic_visit(node)

    def visit_Call(self, node):
        """
        Visit function calls to check for recursion.
        """
        if isinstance(node.func, ast.Name) and node.func.id == self.function_name:
            self.recursive_calls += 1
        self.generic_visit(node)

    def visit_For(self, node):
        """
        Visit for-loops and increment the loop depth.
        """
        self.loop_depth += 1
        if self.loop_depth > self.max_loop_depth:
            self.max_loop_depth = self.loop_depth
        self.generic_visit(node)
        self.loop_depth -= 1

    def visit_While(self, node):
        """
        Visit while-loops and increment the loop depth.
        """
        self.loop_depth += 1
        if self.loop_depth > self.max_loop_depth:
            self.max_loop_depth = self.loop_depth
        self.generic_visit(node)
        self.loop_depth -= 1

    def estimate_complexity(self):
        """
        Estimate the time complexity based on collected data.
        """
        if self.recursive_calls > 0:
            return "O(2^n)"  # Simple heuristic for recursion
        elif self.max_loop_depth > 0:
            return f"O(n^{self.max_loop_depth})"  # Nested loop complexity
        else:
            return "O(1)"  # Constant time if no loops or recursion

def analyze_code_complexity(code):
    """
    Analyzes the given Python code to estimate time complexity.
    
    Parameters:
    - code (str): The Python code as a string.

    Returns:
    - str: Estimated time complexity.
    """
    tree = ast.parse(code)
    visitor = ComplexityVisitor()
    visitor.visit(tree)
    return visitor.estimate_complexity()

# Example usage
if __name__ == "__main__":
    code = """
def bubble_sort(arr):
    n = len(arr)
    for i in range(n):  # Outer loop runs n times
        for j in range(0, n - i - 1):  # Inner loop runs (n - i - 1) times
            if arr[j] > arr[j + 1]:  # Compare adjacent elements
                # Swap if elements are in the wrong order
                arr[j], arr[j + 1] = arr[j + 1], arr[j]
    return arr
    """
    complexity = analyze_code_complexity(code)
    print(f"Estimated Time Complexity: {complexity}")  # Output: O(n^2)

Estimated Time Complexity: O(n^2)


In [2]:
pip install anytree

Note: you may need to restart the kernel to use updated packages.


Master Therorem

In [2]:
def master_theorem(a, b, f_n, n):
    """
    Applies the Master Theorem to solve recurrences of the form T(n) = aT(n/b) + f(n).

    Parameters:
        a (int): The number of subproblems.
        b (int): The factor by which the problem size is reduced.
        f_n (str): The function f(n) as a string (e.g., "n^2", "n log n", "1").
        n (symbol): The symbolic variable n (from SymPy).

    Returns:
        str: The asymptotic solution for T(n).
    """
    import sympy as sp

    # Define the symbolic variable n (if not already defined)
    if not isinstance(n, sp.Symbol):
        n = sp.symbols('n', positive=True, integer=True)

    # Parse f(n) into a symbolic expression
    f_n_expr = sp.sympify(f_n)

    # Compute log_b(a)
    log_b_a = sp.log(a, b)

    # Compare f(n) with n^log_b(a)
    f_n_asymptotic = f_n_expr / (n ** log_b_a)
    f_n_asymptotic = sp.simplify(f_n_asymptotic)

    # Case 1: f(n) = O(n^{log_b(a) - ε}) for some ε > 0
    epsilon = sp.symbols('ε', positive=True)
    if sp.limit(f_n_asymptotic, n, sp.oo) == 0:
        return f"T(n) = Θ(n^{sp.simplify(log_b_a)})"

    # Case 2: f(n) = Θ(n^{log_b(a)} log^k n)
    k = sp.symbols('k', nonnegative=True, integer=True)
    if f_n_asymptotic == sp.log(n) ** k:
        return f"T(n) = Θ(n^{sp.simplify(log_b_a)} log^{k + 1} n)"
    elif f_n_asymptotic == 1:
        return f"T(n) = Θ(n^{sp.simplify(log_b_a)} log n)"

    # Case 3: f(n) = Ω(n^{log_b(a) + ε}) for some ε > 0, and regularity condition
    if sp.limit(f_n_asymptotic, n, sp.oo) == sp.oo:
        # Check regularity condition: a * f(n/b) <= k * f(n) for some k < 1
        regularity_condition = a * f_n_expr.subs(n, n / b) <= sp.Rational(1, 2) * f_n_expr
        if regularity_condition:
            return f"T(n) = Θ({f_n_expr})"

    # If none of the cases apply
    return "Master Theorem does not apply to this recurrence."

In [3]:
import ast
import re

class RecursiveAnalyzer:
    def __init__(self, code):
        self.code = code
        self.variable_changes = {}
        self.recursion_func = []
        self.param_len = 0

    def analyze(self):
        """
        Main function to parse the code and analyze its recursive structure.
        """
        tree = ast.parse(self.code)
        self.traverse_ast(tree)
        self.generate_analysis()

    def traverse_ast(self, node, level=0):
        """
        Traverse the AST and collect information about function parameters,
        recursive calls, and variable changes.
        """
        if isinstance(node, ast.FunctionDef):
            print(f"Function Name: {node.name}")
            params = [arg.arg for arg in node.args.args]
            self.param_len = len(params)
            print(f"Parameters: {', '.join(params)}")

        if isinstance(node, ast.Return):
            return_value = self.print_return_statement(node)
            pattern = r'(\w+\(.*?\))'  # Match recursive function calls
            matches = re.findall(pattern, return_value)
            if matches:
                self.recursion_func.append(matches)

        if isinstance(node, ast.Assign):
            self.handle_assignment(node)

        for child in ast.iter_child_nodes(node):
            self.traverse_ast(child, level + 1)

    def handle_assignment(self, assign_node):
        """
        Handle assignment statements to track variable relationships.
        """
        target = assign_node.targets[0]
        value = assign_node.value

        if isinstance(value, ast.BinOp):
            left = ast.unparse(value.left)
            right = ast.unparse(value.right)
            op = self.get_operator(value.op)
            if isinstance(target, ast.Name):
                self.variable_changes[target.id] = f"{op} {right}"

    def get_operator(self, op):
        """
        Get the operator symbol as a string from the AST node.
        """
        if isinstance(op, ast.Add):
            return "+"
        elif isinstance(op, ast.Sub):
            return "-"
        elif isinstance(op, ast.Mult):
            return "*"
        elif isinstance(op, ast.Div):
            return "/"
        if isinstance(op, ast.FloorDiv):
            return "//"
        elif isinstance(op, ast.Mod):
            return "%"
        return "?"

    def print_return_statement(self, return_node):
        """
        Print and return the return statement of the function.
        """
        return_statement = ast.unparse(return_node.value)
        print(f"Return Statement: {return_statement}")
        return return_statement

    def generate_analysis(self):
        """
        Generate and display analysis of the recursive function.
        """
        print("\n--- Analysis ---")
        
        # Output variable changes
        if self.variable_changes:
            print("Variable Changes:")
            for var, change in self.variable_changes.items():
                print(f"{var}: {change}")

        # Output recursive calls
        print("\nRecursive Calls:")
        if self.recursion_func:
            for i in self.recursion_func:
                print(",".join(i))
        else:
            print("No recursive calls found.")

        # Analyze single vs multiple parameters
        print("\nParameter Analysis:")
        if self.param_len == 1:
            print("Single parameter detected. Likely a single-variable recursion.")
        else:
            print(f"{self.param_len} parameters detected. Likely a multi-variable recursion.")

        # Generate recursive equation
        self.construct_recursive_equation()

    def construct_recursive_equation(self):
        """
        Construct the recursive equation based on identified recursive calls and variable changes.
        """
        if not self.recursion_func:
            print("\nNo recursive calls to construct an equation.")
            return

        equation_parts = []

        if self.param_len == 1 :
            for func_call in self.recursion_func:
                for func in func_call:
                    pattern = r'(\w+)\((.*?)\)'
                    match = re.match(pattern, func)
                    if match:
                        func_name, args = match.groups()
                        args_list = args.split(", ")
                        modified_args = []
                        for arg in args_list:
                            # Check if the argument involves a variable with tracked changes
                            if arg in self.variable_changes:
                                modified_args.append(self.variable_changes[arg])
                            else:
                                modified_args.append(arg)
                        modified_call = f"T({', '.join(modified_args)})"
                        equation_parts.append(modified_call)
        else:
            for var, change in self.variable_changes.items():
                modified_call = f"T({var}{change})"
                equation_parts.append(modified_call)

        # Construct the equation
        recursive_equation = "T(n) = " + " + ".join(equation_parts) + " + O(1)"
        print("\nConstructed Recursive Equation:")
        print(recursive_equation)


if __name__ == "__main__":
    code_fib = """
def fib(n):
    if n == 0 or n == 1:
        return n
    return fib(n-1) + fib(n-2)
"""

    code_binary_search = """
def binary_search(arr, low, high, target):
    if low > high:
        return -1
    mid = (low + high) // 2
    if arr[mid] == target:
        return mid
    elif arr[mid] > target:
        return binary_search(arr, low, mid - 1, target)
    else:
        return binary_search(arr, mid + 1, high, target)
"""

    print("Fibonacci Example:")
    analyzer_fib = RecursiveAnalyzer(code_fib)
    analyzer_fib.analyze()

    print("\nBinary Search Example:")
    analyzer_binary_search = RecursiveAnalyzer(code_binary_search)
    analyzer_binary_search.analyze()


Fibonacci Example:
Function Name: fib
Parameters: n
Return Statement: n
Return Statement: fib(n - 1) + fib(n - 2)

--- Analysis ---

Recursive Calls:
fib(n - 1),fib(n - 2)

Parameter Analysis:
Single parameter detected. Likely a single-variable recursion.

Constructed Recursive Equation:
T(n) = T(n - 1) + T(n - 2) + O(1)

Binary Search Example:
Function Name: binary_search
Parameters: arr, low, high, target
Return Statement: -1
Return Statement: mid
Return Statement: binary_search(arr, low, mid - 1, target)
Return Statement: binary_search(arr, mid + 1, high, target)

--- Analysis ---
Variable Changes:
mid: // 2

Recursive Calls:
binary_search(arr, low, mid - 1, target)
binary_search(arr, mid + 1, high, target)

Parameter Analysis:
4 parameters detected. Likely a multi-variable recursion.

Constructed Recursive Equation:
T(n) = T(mid// 2) + O(1)


In [3]:
master_theorem(1,2,1,'n')

'T(n) = Θ(n^0 log n)'